## **Tarea 2** - Tensorflow: Gradient Descent y Tensorboard

#### Librerias a utilizar

In [157]:
import tensorflow as tf
import numpy as np 
import matplotlib.pyplot as plt
from datetime import datetime

In [175]:
if tf.__version__.startswith("2."):
  import tensorflow.compat.v1 as tf
  tf.compat.v1.disable_v2_behavior()
  tf.compat.v1.disable_eager_execution()

#### Carga de Datos

In [114]:
array = np.load('training_data.npy')

#### Definición de la Clase ***Linear_regression***

In [181]:
class Linear_regression:
    
    def __init__(self):
        tf.reset_default_graph()
        self.parameters_vector = tf.get_variable("parameters", dtype=tf.float32, shape=[2,1], initializer=tf.zeros_initializer())
        
    def cost(self, y_train, y_predict):
        with tf.name_scope("cost"):
            return 1/2*tf.reduce_mean(tf.math.square(y_train - y_predict))
    
    def predict(self, x):
        with tf.name_scope("predict"):
            ones = tf.cast(tf.ones_like(x), tf.float32)
            return tf.matmul(tf.stack((x,ones), axis=-1), self.parameters_vector)
    
    def gradient(self, error_cost, y_train, y_predict):
        return tf.gradients(error_cost, self.parameters_vector)
    
    def update_parameters(self, error_cost, y_train, y_predict, learning_rate):
        with tf.name_scope("gradient"):
            gradient_vector = self.gradient(error_cost, y_train, y_predict)
        with tf.name_scope("update_params"):
            return tf.assign(self.parameters_vector, self.parameters_vector - learning_rate*gradient_vector[0])

#### Definición de la Clase ***Linear_model***

In [179]:
class Linear_model:
    
    def __init__(self, array):
        self.tamano_data_set = len(array)
        self.y = array[:,0]/1000
        self.x = array[:,1]        

    def training(self, learning_rate, steps):
        plt.scatter(x,y)
        lr = Linear_regression()
        x_train = tf.placeholder(tf.float32,[self.tamano_data_set],"x_train")
        y_train = tf.placeholder(tf.float32,[self.tamano_data_set],"y_train")

        y_predict = lr.predict(x)
        error = lr.cost(y_train, y_predict)
        error_summary = tf.summary.scalar(name="errors",tensor=error)
        update_parameters_vector = lr.update_parameters(error, y_train, y_predict, learning_rate)

        with tf.train.MonitoredSession() as session:
            feed_dict = {x_train:x, y_train:y}
            writer = tf.summary.FileWriter("./graphs/"+datetime.now().strftime("%Y%m%d_%H%M%S")+"_lr="+str(learning_rate), session.graph)

            for step in range(steps):
                e_summary = session.run(error_summary, feed_dict=feed_dict)
                session.run(update_parameters_vector,feed_dict=feed_dict)
                prediction = session.run(y_predict,feed_dict=feed_dict)
                writer.add_summary(e_summary, step)
                plt.plot(x,prediction,label=str(step))
            writer.close()

        plt.legend()
        plt.show()

#### Grafo del Modelo

#### Objeto de la clase ***Linear_model***

In [180]:
lm = Linear_model(array)

In [ ]:
lm.training(0.0078, 15)